In [ ]:
!pip install openai langchain faiss-cpu numpy pandas requests tqdm fastapi uvicorn langchain-community


In [3]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [4]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

In [5]:
embedding_model = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))

In [6]:
def load_and_split_document(file_path):
    # Load the document
    loader = PyMuPDFLoader(file_path)
    documents = loader.load()

    # Split the document into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(documents)
    return text_chunks

In [7]:
def initialize_vector_db(text_chunks, embedding_model):
    # Generate embeddings for the text chunks
    vector_db = FAISS.from_documents(text_chunks, embedding_model)
    # Save the vector store for later use
    vector_db.save_local(r"C:\Users\DELL\Downloads\vector_store")
    return vector_db

In [8]:
pip install pymupdf


Note: you may need to restart the kernel to use updated packages.


In [9]:
!pip install tiktoken
!pip install -U langchain-openai faiss-cpu

In [10]:
file_path = r"D:\\advanced\\mccp-java\\Students Encyclopedia of General Knowledge.pdf"
text_chunks = load_and_split_document(file_path)
embedding_model = OpenAIEmbeddings()
vector_db = initialize_vector_db(text_chunks, embedding_model)

In [11]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

In [12]:
def retrieve_and_answer(vector_db, query, llm_model):
    # Create a retrieval-based QA chain
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm_model,
        retriever=vector_db.as_retriever(search_type="similarity", k=1)
    )
    # Process a user query
    return qa_chain.run(query)

In [13]:
!pip install langchain-experimental


In [14]:
from langchain_experimental.agents import create_csv_agent
from langchain_openai import OpenAI


In [15]:
from langchain_experimental.agents import create_csv_agent
from langchain_openai import ChatOpenAI
def summarize_text(text_chunk):
    # Initialize the LLM model for summarization
    model = ChatOpenAI(model="gpt-3.5-turbo")  # or "gpt-4"

    # Define the prompt as a message for the chat model
    messages = [
        {"role": "system", "content": "You are an expert summarizer."},
        {"role": "user", "content": f"Summarize the following text:\n{text_chunk}"}
    ]

    # Generate the summary using `invoke`
    response = model.invoke(messages)
    
    # Inspect the response type and content
    summary = response.content.strip()  # Access `content` attribute directly
    return summary






In [16]:
# Example usage
# Sample usage within the main function or a script
llm_model = ChatOpenAI(model="gpt-3.5-turbo")
query = "ATM stands for?"
answer = retrieve_and_answer(vector_db, query, llm_model)
summary = summarize_text(answer)
print(summary)


C:\Users\DELL\AppData\Local\Temp\ipykernel_8548\4285666998.py:8: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  return qa_chain.run(query)


An Automated Teller Machine (ATM) is a machine that allows users to perform basic banking transactions, such as withdrawing and depositing cash, checking account balances, and transferring funds.


In [ ]:
pip install --upgrade langsmith


In [29]:
pip install - U langsmith

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '-': Expected package name at the start of dependency specifier
    -
    ^


In [30]:
from langsmith import Client as LangSmithClient

In [31]:
from openai import OpenAI
from langsmith import wrappers

client = wrappers.wrap_openai(OpenAI())

In [21]:
client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Say this is a test"}],
)

ChatCompletion(id='chatcmpl-A20sTconp4ohfFjpLm29XrDzc7vnU', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='This is a test.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1725043393, model='gpt-4-0613', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=5, prompt_tokens=12, total_tokens=17))

In [22]:
from langsmith import traceable

@traceable
def my_function(text: str):
    return client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": f"Say {text}"}],
    )

my_function("hello world")

ChatCompletion(id='chatcmpl-A20sVTqNCitEq5F0O55n14sAWTSkn', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello world!', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1725043395, model='gpt-4-0613', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=3, prompt_tokens=10, total_tokens=13))

In [52]:
import os
from langsmith import Client

def track_interaction(query, response):
    # Retrieve API key from environment variable
    api_key = os.getenv("LANGSMITH_API_KEY")
    if not api_key:
        raise ValueError("API key for LangSmith is missing.")
    
    # Initialize the LangSmith Client with the API key
    client = Client(api_key=api_key)
    
    try:
        # Log interaction as an example
        client.create_example(prompt=query, completion=response)  # Replace with actual parameters if needed
    except AttributeError:
        print("The 'Client' object does not have the expected method. Please check the LangSmith API documentation.")
    except Exception as e:
        print(f"Failed to log interaction: {e}")

# Example usage
query = "What are the benefits of regular exercise?"
response = "Regular exercise has many benefits including improved cardiovascular health, better mood, and increased longevity."
track_interaction(query, response)


Failed to log interaction: Exactly one argument in each of the following groups must be defined: dataset_id, dataset_name


In [51]:
import os
from langsmith import Client

# Initialize LangSmith Client
api_key = os.getenv("LANGSMITH_API_KEY")  # Replace with actual API key
client = Client(api_key=api_key)

# Print available methods and attributes
print(dir(client))

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_anonymizer', '_create_run', '_current_tenant_is_owner', '_filter_for_sampling', '_get_cursor_paginated_list', '_get_data_type', '_get_data_type_cached', '_get_latest_commit_hash', '_get_optional_tenant_id', '_get_paginated_list', '_get_prompt_url', '_get_settings', '_get_tenant_id', '_headers', '_hide_inputs', '_hide_outputs', '_hide_run_inputs', '_hide_run_outputs', '_host', '_host_url', '_info', '_insert_runtime_env', '_like_or_unlike_prompt', '_load_child_runs', '_log_evaluation_feedback', '_owner_conflict_error', '_post_batch_ingest_runs', '_prompt_exists', '_repr_html_', '_resolve_example_id', '_resolve_run_id', '_run_trans

In [39]:
import logging

# Configure logging
logging.basicConfig(filename='interactions.log', level=logging.INFO, format='%(asctime)s - %(message)s')

def track_interaction(query, response):
    # Log the interaction
    logging.info(f"Query: {query}\nResponse: {response}")

# Example usage
query = "What are the benefits of regular exercise?"
response = retrieve_and_answer(vector_db, query, llm_model)
track_interaction(query, response)